In [2]:
# Part 1 : gensim LDA based on NLTK & SpaCy

# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tharsen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# For Shakespeare : We need to change this: kill thou, thy and shall & keep subject, re, edu and use
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(['from', 'thou', 'thy', 'shall', 'would'])  # make, come, go also very common


In [5]:
# Import Dataset -- original source
'''
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])
'''

'\ndf = pd.read_json(\'https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json\')\nprint(df.target_names.unique())\ndf.head()\n\n# Convert to list\ndata = df.content.values.tolist()\n\n# Remove Emails\ndata = [re.sub(\'\\S*@\\S*\\s?\', \'\', sent) for sent in data]\n\n# Remove new line characters\ndata = [re.sub(\'\\s+\', \' \', sent) for sent in data]\n\n# Remove distracting single quotes\ndata = [re.sub("\'", "", sent) for sent in data]\n\npprint(data[:1])\n'

In [8]:
# Import Dataset -- Shakespeare

# Pull all into "data" = all Shakespeare raw text

# directory containing all source texts for training the model 
data_dir="/Users/tharsen/Desktop/populism-rcc-hackathon/corpus/"

import glob, os
os.chdir(data_dir)

#documents = list()
data = list()   # reset data to 0

for filename in glob.glob("*.txt"):
    filedata = open(filename, 'r').read()
    print(filename + " = " + str(len(filedata)) + " chars")
    #documents = documents + filedata.split(".")
    data.append(filedata)


1H4_h.txt = 130423 chars
1H6_h.txt = 116317 chars
2H4_h.txt = 141574 chars
2H6_h.txt = 135863 chars
3H6_h.txt = 129551 chars
Ado_c.txt = 111116 chars
Ant_t.txt = 133668 chars
AWW_c.txt = 121896 chars
AYL_c.txt = 114429 chars
Cor_t.txt = 146443 chars
Cym_t.txt = 147159 chars
Err_c.txt = 76924 chars
H5_h.txt = 141819 chars
H8_h.txt = 128223 chars
Ham_t.txt = 163429 chars
JC_t.txt = 104561 chars
John_t.txt = 112414 chars
Lear_t.txt = 140510 chars
LLL_c.txt = 115391 chars
Lucrece_x.txt = 86177 chars
M4M_c.txt = 116348 chars
Mac_t.txt = 91625 chars
MerchV_c.txt = 112334 chars
MND_c.txt = 88608 chars
Oth_t.txt = 141395 chars
Pericles_x.txt = 97471 chars
PhxTur_x.txt = 2072 chars
R2_h.txt = 120934 chars
R3_h.txt = 156881 chars
Rom_t.txt = 130885 chars
Shr_c.txt = 111364 chars
Sonnets_x.txt = 97204 chars
TGV_c.txt = 91686 chars
Tim_t.txt = 98749 chars
Tit_t.txt = 109892 chars
Tmp_c.txt = 88800 chars
TN_c.txt = 104476 chars
TNK_x.txt = 127691 chars
Tro_c.txt = 142635 chars
VenusAdonis_x.txt = 5

In [6]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])
print('\n')

[['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care', 'find', 'we', 'time', 'for', 'frighted', 'peace', 'to', 'pant', 'and', 'breathe', 'short', 'winded', 'accents', 'of', 'new', 'broils', 'to', 'be', 'commenced', 'in', 'strands', 'afar', 'remote', 'no', 'more', 'the', 'thirsty', 'entrance', 'of', 'this', 'soil', 'shall', 'daub', 'her', 'lips', 'with', 'her', 'own', 'children', 'blood', 'no', 'more', 'shall', 'trenching', 'war', 'channel', 'her', 'fields', 'nor', 'bruise', 'her', 'flow', 'rets', 'with', 'the', 'armed', 'hoofs', 'of', 'hostile', 'paces', 'those', 'opposed', 'eyes', 'which', 'like', 'the', 'meteors', 'of', 'troubled', 'heaven', 'all', 'of', 'one', 'nature', 'of', 'one', 'substance', 'bred', 'did', 'lately', 'meet', 'in', 'the', 'intestine', 'shock', 'and', 'furious', 'close', 'of', 'civil', 'butchery', 'shall', 'now', 'in', 'mutual', 'well', 'beseeming', 'ranks', 'march', 'all', 'one', 'way', 'and', 'be', 'no', 'more', 'opposed', 'against', 'acquaintance', 'k

In [7]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
print('\n')

['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care', 'find', 'we', 'time', 'for', 'frighted', 'peace', 'to', 'pant', 'and', 'breathe', 'short', 'winded', 'accents', 'of', 'new', 'broils', 'to', 'be', 'commenced', 'in', 'strands', 'afar', 'remote', 'no', 'more', 'the', 'thirsty', 'entrance', 'of', 'this', 'soil', 'shall', 'daub', 'her', 'lips', 'with', 'her', 'own', 'children', 'blood', 'no', 'more', 'shall', 'trenching', 'war', 'channel', 'her', 'fields', 'nor', 'bruise', 'her', 'flow', 'rets', 'with', 'the', 'armed', 'hoofs', 'of', 'hostile', 'paces', 'those', 'opposed', 'eyes', 'which', 'like', 'the', 'meteors', 'of', 'troubled', 'heaven', 'all', 'of', 'one', 'nature', 'of', 'one', 'substance', 'bred', 'did', 'lately', 'meet', 'in', 'the', 'intestine', 'shock', 'and', 'furious', 'close', 'of', 'civil', 'butchery', 'shall', 'now', 'in', 'mutual', 'well', 'beseeming', 'ranks', 'march', 'all', 'one', 'way', 'and', 'be', 'no', 'more', 'opposed', 'against', 'acquaintance', 'ki

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
%%time
# This cell takes 2-3 minutes to run on my machine.  -j

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print('\n')

[['shake', 'wan', 'care', 'find', 'time', 'fright', 'peace', 'pant', 'breathe', 'short', 'wind', 'accent', 'new', 'broil', 'commence', 'strand', 'afar', 'remote', 'thirsty', 'entrance', 'soil', 'daub', 'lip', 'child', 'blood', 'trench', 'war', 'channel', 'field', 'bruise', 'flow', 'ret', 'armed', 'hoof', 'hostile', 'pace', 'oppose', 'eye', 'meteor', 'trouble', 'nature', 'substance', 'breed', 'lately', 'meet', 'intestine', 'shock', 'furious', 'close', 'civil', 'butchery', 'mutual', 'well', 'beseem', 'rank', 'march', 'way', 'oppose', 'acquaintance', 'kindr', 'ally', 'edge', 'war', 'ill', 'sheathed', 'knife', 'cut', 'master', 'therefore', 'friend', 'far', 'sepulcher', 'christ', 'whose', 'soldier', 'whose', 'bless', 'cross', 'impressed', 'engage', 'fight', 'forthwith', 'power', 'english', 'levy', 'whose', 'arm', 'mold', 'mother', 'womb', 'chase', 'pagan', 'holy', 'field', 'whose', 'acre', 'walk', 'bless', 'foot', 'year', 'ago', 'nail', 'advantage', 'bitter', 'cross', 'purpose', 'month', 'o

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 2), (6, 1), (7, 1), (8, 2), (9, 1), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 4), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 4), (23, 1), (24, 2), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 7), (32, 4), (33, 1), (34, 2), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 8), (46, 1), (47, 4), (48, 1), (49, 1), (50, 1), (51, 6), (52, 1), (53, 4), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 3), (61, 1), (62, 1), (63, 1), (64, 1), (65, 9), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 9), (72, 1), (73, 1), (74, 11), (75, 2), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 3), (86, 1), (87, 2), (88, 1), (89, 4), (90, 4), (91, 4), (92, 1), (93, 1), (94, 1), (95, 1), (96, 6), (97, 1), (98, 15), (99, 1), (100, 1), (101, 1), (102, 4), (103, 1), (104, 1), (105, 1), (106, 2), (107, 2), (108, 1), (109, 1), (110, 

In [11]:
id2word[0]

'abhor'

In [12]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abhor', 1),
  ('able', 1),
  ('abominable', 1),
  ('abroad', 1),
  ('absence', 3),
  ('absent', 2),
  ('absolute', 1),
  ('abundance', 1),
  ('abuse', 2),
  ('accent', 1),
  ('accept', 2),
  ('accident', 1),
  ('accompany', 2),
  ('accord', 1),
  ('accordingly', 1),
  ('account', 4),
  ('accursed', 1),
  ('accusation', 1),
  ('achieve', 1),
  ('acknowledge', 1),
  ('acquaintance', 2),
  ('acre', 1),
  ('action', 4),
  ('active', 1),
  ('adam', 2),
  ('addition', 1),
  ('adieu', 2),
  ('admiral', 1),
  ('admire', 1),
  ('ado', 1),
  ('adopt', 1),
  ('advantage', 7),
  ('adventure', 4),
  ('adventurous', 1),
  ('adversary', 2),
  ('advertisement', 2),
  ('advise', 1),
  ('advisedly', 1),
  ('afar', 1),
  ('afeard', 1),
  ('affable', 1),
  ('affair', 1),
  ('affection', 1),
  ('afford', 1),
  ('affright', 1),
  ('afoot', 8),
  ('afore', 1),
  ('afraid', 4),
  ('afront', 1),
  ('afternoon', 1),
  ('agate', 1),
  ('age', 6),
  ('agent', 1),
  ('ago', 4),
  ('agree', 1),
  ('agreement', 

In [13]:
%%time

### Much as we saw with Word2vec, these settings are the key to tuning your LDA Topic Model. ###

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 26.8 s, sys: 8.33 s, total: 35.2 s
Wall time: 9.63 s


In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.001*"come" + 0.001*"make" + 0.001*"man" + 0.001*"go" + 0.001*"good" + '
  '0.001*"love" + 0.001*"know" + 0.001*"sir" + 0.001*"lord" + 0.001*"may"'),
 (1,
  '0.035*"valentine" + 0.035*"sylvia" + 0.032*"proteus" + 0.020*"julia" + '
  '0.014*"thurio" + 0.008*"love" + 0.007*"item" + 0.006*"eglamour" + '
  '0.004*"lucetta" + 0.003*"ladyship"'),
 (2,
  '0.002*"lord" + 0.001*"man" + 0.001*"come" + 0.001*"good" + 0.001*"let" + '
  '0.001*"make" + 0.001*"go" + 0.001*"say" + 0.001*"see" + 0.001*"know"'),
 (3,
  '0.001*"love" + 0.001*"come" + 0.001*"let" + 0.001*"good" + 0.001*"make" + '
  '0.001*"say" + 0.001*"go" + 0.001*"man" + 0.001*"know" + 0.001*"sir"'),
 (4,
  '0.002*"come" + 0.002*"make" + 0.002*"good" + 0.001*"man" + 0.001*"go" + '
  '0.001*"sir" + 0.001*"know" + 0.001*"say" + 0.001*"king" + 0.001*"love"'),
 (5,
  '0.023*"caesar" + 0.015*"brutus" + 0.013*"man" + 0.011*"come" + 0.009*"good" '
  '+ 0.008*"know" + 0.008*"cassius" + 0.008*"let" + 0.007*"go" + 0.006*"say"'),
 (6,
  

In [15]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\n')


Perplexity:  -8.00860722412174

Coherence Score:  0.2903802007810519


CPU times: user 2.77 s, sys: 1.05 s, total: 3.83 s
Wall time: 16.1 s


In [16]:
%%time

# Visualize the topics
#
# If you get an error like this: "pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. 
# A future version of pandas will change to not sort by default."
#
# then from the command line do: "pip install pandas==0.21.0"

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)  # sort=False ? sort=True
vis
print('\n')



CPU times: user 1.6 s, sys: 1.02 s, total: 2.62 s
Wall time: 4.44 s


In [17]:
#import pandas as pd
#pd.__version__

vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
12     67.770607        1       1 -0.169607  0.003002
9      22.309572        1       2 -0.213568 -0.011112
15      4.322424        1       3 -0.223944 -0.043753
10      2.284129        1       4 -0.191145 -0.034620
5       2.179936        1       5 -0.197165  0.064127
6       0.561118        1       6 -0.071385  0.128110
8       0.450951        1       7 -0.003581 -0.139540
1       0.071084        1       8  0.083465  0.000665
18      0.047866        1       9  0.054335 -0.001591
19      0.000226        1      10  0.065992  0.001163
4       0.000225        1      11  0.067372  0.004309
14      0.000218        1      12  0.079165  0.002678
11      0.000214        1      13  0.080758  0.001014
3       0.000207        1      14  0.089126  0.003282
2       0.000207        1      15  0.086524  0.003623
0       0.000207        1      16  0.089271  0.004294
7       0.000205        1      17  0.082448  0.000962
13      0.000203        1      18  0.090154  0.004304
16      0.000200        1      19  0.093805  0.004088
17      0.000197        1      20  0.107981  0.004996, topic_info=     Category         Freq   Term        Total  loglift  logprob
term                                                            
494   Default  3771.000000   come  3771.000000  30.0000  30.0000
1102  Default  3601.000000   good  3601.000000  29.0000  29.0000
1545  Default  3116.000000   lord  3116.000000  28.0000  28.0000
1553  Default  2931.000000   love  2931.000000  27.0000  27.0000
1588  Default  3158.000000   make  3158.000000  26.0000  26.0000
1593  Default  3051.000000    man  3051.000000  25.0000  25.0000
1419  Default  1649.000000   king  1649.000000  24.0000  24.0000
1096  Default  2620.000000     go  2620.000000  23.0000  23.0000
1487  Default  2529.000000    let  2529.000000  22.0000  22.0000
2330  Default  2615.000000    sir  2615.000000  21.0000  21.0000
1432  Default  2494.000000   know  2494.000000  20.0000  20.0000
2222  Default  2482.000000    say  2482.000000  19.0000  19.0000
1619  Default  2350.000000    may  2350.000000  18.0000  18.0000
2260  Default  2200.000000    see  2200.000000  17.0000  17.0000
1085  Default  2068.000000   give  2068.000000  16.0000  16.0000
1196  Default  2045.000000   hath  2045.000000  15.0000  15.0000
2853  Default  2032.000000   well  2032.000000  14.0000  14.0000
1716  Default  1624.000000   must  1624.000000  13.0000  13.0000
2542  Default  1657.000000   take  1657.000000  12.0000  12.0000
2391  Default  1613.000000  speak  1613.000000  11.0000  11.0000
2598  Default  1583.000000  think  1583.000000  10.0000  10.0000
889   Default  1296.000000    eye  1296.000000   9.0000   9.0000
2627  Default  1333.000000   time  1333.000000   8.0000   8.0000
2567  Default  1453.000000   tell  1453.000000   7.0000   7.0000
2630  Default  1302.000000    tis  1302.000000   6.0000   6.0000
1205  Default  1375.000000   hear  1375.000000   5.0000   5.0000
1207  Default  1304.000000  heart  1304.000000   4.0000   4.0000
1541  Default  1273.000000   look  1273.000000   3.0000   3.0000
2940  Default  1178.000000    yet  1178.000000   2.0000   2.0000
742   Default  1070.000000   doth  1070.000000   1.0000   1.0000
...       ...          ...    ...          ...      ...      ...
1593  Topic20     0.000768    man  3051.609131  -2.0594  -6.9268
1102  Topic20     0.000744   good  3601.139404  -2.2568  -6.9587
2330  Topic20     0.000659    sir  2615.779053  -2.0591  -7.0806
494   Topic20     0.000720   come  3771.356689  -2.3364  -6.9920
1487  Topic20     0.000596    let  2529.373291  -2.1256  -7.1807
1545  Topic20     0.000631   lord  3116.418701  -2.2780  -7.1244
1096  Topic20     0.000571     go  2620.567871  -2.2035  -7.2231
2260  Topic20     0.000511    see  2200.073730  -2.1396  -7.3342
2627  Topic20     0.000405   time  1333.580444  -1.8712  -7.5664
1588  Topic20     0.000507   make  3158